# MARCS Common Blocks

Identifying Fortran common blocks used throughout the MARCS model atmosphere package. The goal is to have a list of common blocks with an index of each routine they appear in.

In [1]:
import fileinput as fi

I have already run `grep` from the command line using
```bash
grep -i -n "common" *.f > marcs_common_blocks.txt
```
The option `-i` indicates that the search should be case insensitive and `-n` returns the line number on which the search phrase is used. I've also piped the output to a file called `marcs_common_blocks.txt` for easy manipulation. All MARCS files have the `.f` Fortran extension, so all instances should be returned using this search.

Now, let's look at the file structure.

In [2]:
!head -n 5 marcs_common_blocks.txt

algebn.f:50:      COMMON/SPACE1/A1,A2,A3,D1,D2,B1,B2,B3,C,Q1,O1,
archiv.f:23:C        COMMONS SHARED BY MAIN PROGRAM
archiv.f:25:      COMMON /TAUC/TAU(NDP),DTAULN(NDP),JTAU
archiv.f:26:      COMMON/STATEC/PRAD(NDP),PTURB(NDP),P(NDP),GG(NDP),ZZ(NDP),DD(NDP),
archiv.f:29:      COMMON /ROSSC/XKAPR(NDP),CROSS(NDP)


The basic structure is `filename.f:##:` followed by the contents on the line. Since older Fortran required users to start in the 7th column, there is ample whitespace between the file information and the line content. The only exception is when a line is commented out. 

We can read the data in and separate it using the colon, `:`, as a delimeter.

In [3]:
marcs_common_blocks = [line.split(':') for line in fi.input('marcs_common_blocks.txt')]

Check to make sure we've acheived what we set out to do.

In [4]:
marcs_common_blocks[0]

['algebn.f', '50', '      COMMON/SPACE1/A1,A2,A3,D1,D2,B1,B2,B3,C,Q1,O1,\n']

Now we need to figure out whether we can easily access common block names. They are always surrounded by `/ /`, but we need to be careful to avoid irregular spacings. It is therefore advantageous to trim all whitespace in the third column before populating the list. We also want to strip new line characters and convert everything to lower case. However, let us also avoid commented lines and focus only on active common blocks. Comments are indicated by either `c`, `!`, or `* `.

In [5]:
common_block_names = [entry[2].rstrip('\n').lower().replace(' ', '') 
                      for entry in marcs_common_blocks if entry[2][0].lower() not in ['c', '!', '*']]

With commented entries removed, all common blocks can be identified by their initial `c` character. This will ensure that all unwanted entries that spuriously ended up in the list are removed. Then, we extract common block names by looking what is between the `/ /`. 

In [6]:
common_block_names = [entry[entry.find('/') + 1:entry.rfind('/')] for entry in common_block_names 
                      if entry[0].lower() == 'c']

Check whether we've isolated common block names.

In [7]:
common_block_names[0], common_block_names[50], common_block_names[-1]

('space1', 'density', 'ldopac')

Remove duplicates from the list.

In [8]:
common_block_names = list(set(common_block_names))

Here's a full listing.

In [9]:
common_block_names.sort()
common_block_names

['auxabund',
 'auxabund/absc,abti,abv,abmn,abco/auxabund',
 'babsma',
 'bplc',
 'c030617',
 'ca1',
 'ca1/delt,tbot,idel,isvit,iteta,kvadt,/ca1',
 'ca2',
 'ca3',
 'ca4',
 'ca5',
 'ca5/ab,fakt,pe,t,xla,xla3,ro,sumabs,sumsca,viktr,iset,nlb/ca5',
 'cangle',
 'carc1',
 'carc2',
 'carc2c',
 'carc3',
 'carc4',
 'carc4/prov,nprova,nprovs,nprov/carc4',
 'catfil',
 'ccontmolc',
 'cemu',
 'cfil',
 'cfil/ireset,islask,ireat/cfil',
 'cg',
 'cg/grav/cteff',
 'char',
 'ci1',
 'ci3',
 'ci4',
 'ci5',
 'ci5/abund,anjon,h,part,dxi,f1,f2,f3,f4,f5,xkhm,xmh,xmy/ci5',
 'ci6',
 'ci7',
 'ci8',
 'ci9',
 'cit',
 'cline1',
 'cline2',
 'cline3',
 'cline4',
 'cline4/iline/cline4',
 'clockmess',
 'cmetbl',
 'cmol1',
 'cmol2',
 'comfh1',
 'coutr',
 'coutr/nto,ntpo/coutr',
 'cpf',
 'cphydro',
 'cpoly',
 'cresume',
 'cros',
 'cros/wros/cros',
 'cspher',
 'cspher/ncore,diflog,radius,rr(ndp)/ctaum',
 'cstyr',
 'cstyr/mihal,noconv/ctaum',
 'cstyr/mihal,noconv/cxmax/xmax/ctaum',
 'cstyr/mihal,noconv/debug',
 'csurf',
 'cte

Thre are clearly some issues related to programming styles. Most repeated occurrences are the result of the user "closing" the common block or by including multiple common blocks on a single line. Let's remove those with some brute force tactics.

In [10]:
second_round_names = [entry[entry.rfind('/') + 1:] for entry in common_block_names if entry.rfind('/') != -1]

In [11]:
second_round_names

['auxabund',
 'ca1',
 'ca5',
 'carc4',
 'cfil',
 'cteff',
 'ci5',
 'cline4',
 'coutr',
 'cros',
 'ctaum',
 'ctaum',
 'ctaum',
 'debug',
 'cg',
 'cvaagl',
 'cxlset',
 'ldopac',
 'cstyr',
 'cvfix',
 'odfwav',
 'rhoc',
 'rhoc',
 'utput']

Only one entry has three common block names, but luckily the third name is already indexed, so we can move on. Get only the first common block name from the original list.

In [12]:
first_round_names = [entry for entry in common_block_names if entry.rfind('/') == -1]

Combine the two lists and remove duplicate entries.

In [13]:
common_block_names = list(set(first_round_names + second_round_names))

In [14]:
common_block_names.sort()

Now we are in a position to create a table of contents for our common blocks. It may be best for visualization if we write it in both plain text and markdown. 

First, a test to get a proper formatting.

In [15]:
key = common_block_names[0]
print key.upper()
for entry in marcs_common_blocks:
    if entry[2].lower().find(key) != -1:
        print "\t {:16s} on line: {:4s}".format(entry[0], entry[1])

AUXABUND
	 archiv.f         on line: 71  
	 eqmol_pe.f       on line: 102 
	 eqmol_pe_lu.f    on line: 102 
	 injon.f          on line: 108 


That seems to be quite reasonable. Now for all keys,

In [16]:
for key in common_block_names:
    print key.upper()
    for entry in marcs_common_blocks:
        if entry[2].lower().find(key) != -1:
            print "\t {:16s} on line: {:4s}".format(entry[0], entry[1])
        else:
            pass

AUXABUND
	 archiv.f         on line: 71  
	 eqmol_pe.f       on line: 102 
	 eqmol_pe_lu.f    on line: 102 
	 injon.f          on line: 108 
BABSMA
	 oldsta.f         on line: 24  
	 oldsta_test.f    on line: 20  
BPLC
	 bpl.f            on line: 4   
C030617
	 archiv.f         on line: 97  
	 detabs.f         on line: 119 
	 osabsko.f        on line: 65  
CA1
	 inabs.f          on line: 96  
	 osinit.f         on line: 21  
	 tabs.f           on line: 20  
	 tabs.f           on line: 36  
CA2
	 detabs.f         on line: 117 
	 inabs.f          on line: 99  
	 osabsko.f        on line: 31  
	 osabsko.f        on line: 49  
	 tabs.f           on line: 39  
CA3
	 inabs.f          on line: 100 
	 osabsko.f        on line: 31  
	 osabsko.f        on line: 50  
CA4
	 osabsko.f        on line: 32  
	 osabsko.f        on line: 51  
	 tabs.f           on line: 40  
CA5
	 detabs.f         on line: 56  
	 detabs.f         on line: 90  
	 osabsko.f        on line: 52  
	 tryck.f          on line:

That clearly works, so let's output that information to a plain text file.

In [17]:
plaint = open('common_block_index.txt', 'w')
for key in common_block_names:
    plaint.write(key.upper() + '\n')
    for entry in marcs_common_blocks:
        if entry[2].lower().find(key) != -1:
            plaint.write("\t {:30s} on line: {:4s} \n".format(entry[0], entry[1]))
        else:
            pass
    plaint.write('\n')
plaint.close()

And in markdown for easy reading online.

In [18]:
markd = open('common_block_index.md', 'w')
for key in common_block_names:
    markd.write('## ' + key.upper() + '\n')
    for entry in marcs_common_blocks:
        if entry[2].lower().find(key) != -1:
            markd.write("\t {:30s} on line: {:4s} \n".format(entry[0], entry[1]))
        else:
            pass
    markd.write('\n')
markd.close()